
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.


In [2]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-martynadurda')


Successfully saved authorization token.



## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [3]:
aoi = ee.Geometry.Point([19.9449799, 50.0646501])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [4]:
image = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
         .filterBounds(aoi)
         .filterDate('2021-07-01', '2021-07-31')
         .sort('CLOUD_COVER')
         .first())

print('Image ID:', image.getInfo()['id'])

Image ID: LANDSAT/LC08/C02/T1/LC08_188025_20210728



## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [5]:
def radiance_band(band):
    M = ee.Number(image.get(f'RADIANCE_MULT_BAND_{band}'))
    A = ee.Number(image.get(f'RADIANCE_ADD_BAND_{band}'))
    DN = image.select(f'B{band}')
    return DN.multiply(M).add(A).rename(f'B{band}_rad')

radiance = ee.Image.cat([
    radiance_band(2),
    radiance_band(3),
    radiance_band(4),
    radiance_band(5)
])


## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [8]:
import math

# ESUN values for each band
ESUN = {
    'B2': 2067,
    'B3': 1893,
    'B4': 1603,
    'B5': 972
}


sun_elevation = ee.Number(image.get('SUN_ELEVATION'))
theta_s = ee.Number(90).subtract(sun_elevation).multiply(math.pi).divide(180)


def reflectance_band(band):
    L = radiance.select(f'B{band}_rad')
    esun = ESUN[f'B{band}']
    reflectance = L.multiply(math.pi).divide(ee.Number(esun).multiply(theta_s.cos())).rename(f'B{band}_refl')
    return reflectance

reflectance = ee.Image.cat([
    reflectance_band(2),
    reflectance_band(3),
    reflectance_band(4),
    reflectance_band(5)
])


## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [7]:
Map = geemap.Map(center=[50.0647, 19.945], zoom=10)

# Original DN RGB visualization
dn_vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 30000,
    'gamma': 1.4
}
Map.addLayer(image, dn_vis_params, 'Original DN RGB')

# TOA Reflectance visualization (clamped to [0, 0.4] for better contrast)
toa_rgb = reflectance.select(['B4_refl', 'B3_refl', 'B2_refl']).clamp(0, 0.4)
toa_vis_params = {
    'min': 0,
    'max': 0.4
}
Map.addLayer(toa_rgb, toa_vis_params, 'TOA Reflectance RGB')

Map.addLayerControl()
Map

Map(center=[50.0647, 19.945], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
